In [9]:
m1 = "WR NOIVBIG LVJCFW JCL NFOWMZZL XSZOMUFLVW OOTO GSQBO GQ BCFBOF TCSGWI GSUI"
m2 = "BHTZS FIHOFWAPH LOJDVHW DTZO VIBHD ZSQLBO ZD KVWDDS"

In [10]:
def vigenere_decrypt(ciphertext, key):
    """
    Decifra un testo cifrato con il cifrario di Vigenère mantenendo spazi e punteggiatura.
    
    Args:
        ciphertext (str): testo cifrato
        key (str): chiave per la decifratura
    
    Returns:
        str: testo in chiaro
    """
    key = key.upper()
    plaintext = []
    key_index = 0

    for c in ciphertext:
        if c.isalpha():
            c_upper = c.upper()
            c_index = ord(c_upper) - ord('A')
            k_index = ord(key[key_index % len(key)]) - ord('A')
            p_index = (c_index - k_index) % 26
            p_char = chr(p_index + ord('A'))

            # Mantieni maiuscola/minuscola originale
            if c.islower():
                p_char = p_char.lower()
            plaintext.append(p_char)

            key_index += 1
        else:
            # Mantieni spazi e punteggiatura
            plaintext.append(c)

    return ''.join(plaintext)


In [11]:
vigenere_decrypt(m1, "ODIO")

'IO FAUSTUS INVOCO VOI FRATELLI PELLEGRINI ALLA SPINA DI NOCTAR QUESTA SERA'

In [12]:
vigenere_decrypt(m2, "ODIO")

'NELLE CATACOMBE DAVANTI ALLA SANTA RECITA LA CHIAVE'

In [5]:
import json

with open('./results/test_name/4250088.json', 'r') as f:
    a = json.loads(f.read())

In [6]:
a.keys( )

dict_keys(['metadata', 'data'])

In [7]:
len(a['data'])

11